In [248]:
import os
import sys
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [249]:
results_dir = './results'

In [258]:
results_files = list()
overall_results_df = None
results_files_columns = ['method', 'model', 'system', 'prompt', 'temperature', 'run', 'dataset']

for results_file in sorted(os.listdir(results_dir)):
    if results_file.endswith('overall.csv') or results_file.endswith('granular.csv'):
        continue
    
    results_file_parts = results_file.split('.csv')[0].split('_')
    
    system, prompt, temperature, run = ['NA']*4

    if results_file_parts[0] == 'code-llama-34b':
        method = 'LLM'
        model = results_file_parts[0]
        system, prompt, temperature = results_file_parts[1:4]
        dataset = results_file_parts[4]
        run = results_file_parts[5]
    elif results_file_parts[0] == 'OpenAI':
        method = 'LLM'
        model = results_file_parts[1]
        system, prompt, temperature = results_file_parts[2:5]
        dataset = results_file_parts[5]
        run = results_file_parts[6]
    elif results_file_parts[0] == 'exact':
        method = results_file_parts[0]
        model = 'exact'
        dataset = results_file_parts[1]
    elif results_file_parts[0] == 'deepcadrme':
        method = 'BERT'
        model = 'deepcadrme'
        system = results_file_parts[1]
        dataset = results_file_parts[2]
    elif results_file_parts[0] in ('granular', 'overall'):
        continue
    else:
        raise Exception(results_file_parts[0])

    results_files.append([method, model, system, prompt, temperature, run, dataset])

    for evaltype in ('strict', 'lenient'):
        overall_results_file = results_file.replace('.csv', f"_{evaltype}_overall.csv")
        if not os.path.exists(os.path.join(results_dir, overall_results_file)):
            continue
        res = pd.read_csv(os.path.join(results_dir, overall_results_file), index_col=0)
        for name, colval in zip(results_files_columns, results_files[-1]):
            res.insert(0, name, [colval]*res.shape[0])
        res.insert(0, 'evaltype', [evaltype]*res.shape[0])
        
        if overall_results_df is None:
            overall_results_df = res
        else:
            overall_results_df = pd.concat([overall_results_df, res])

results_files_df = pd.DataFrame(results_files, columns=results_files_columns)

In [259]:
results_files_df.groupby(['dataset', 'method', 'model', 'system', 'prompt', 'temperature']).count()

run
dataset method model              system          prompt           temperature     
test    BERT   deepcadrme         002             NA               NA             1
                                  005             NA               NA             1
                                  010             NA               NA             1
                                  015             NA               NA             1
                                  025             NA               NA             1
                                  050             NA               NA             1
                                  075             NA               NA             1
                                  100             NA               NA             1
        LLM    gpt-3.5-turbo-0125 fatal-prompt-v2 pharmexpert-v1   temp0          3
               gpt-4-1106-preview fatal-prompt-v2 pharmexpert-v1   temp0          2
                                  fatal-prompt-v3 pharmexpert-v1   temp0          1
        exact  exact              NA              NA               NA             1
train   LLM    code-llama-34b     fatal-prompt-v2 pharmexpert-v0   temp0          1
                                                  pharmexpert-v1   temp0          1
               gpt-3.5-turbo-0125 fatal-prompt-v2 no-system-prompt temp0          3
                                                  pharmexpert-v0   temp0          3
                                                                   temp0.5        3
                                                  pharmexpert-v1   temp0          4
               gpt-4-1106-preview fatal-prompt-v2 no-system-prompt temp0          3
                                                  pharmexpert-v0   temp0          3
                                                                   temp0.5        3
                                                                   temp1.0        3
                                                                   temp1.5        1
                                                  pharmexpert-v1   temp0          5
                                  fatal-prompt-v3 pharmexpert-v1   temp0          1
        exact  exact              NA              NA               NA             1

In [267]:
# default eval
groupbycols = ['dataset', 'ade_type', 'section', 'method', 'model', 'system', 'prompt', 'temperature']
additional_querystr = " & ade_type=='all'"

# evaluate the prompt
# found that pharmexpert-v1 is working a bit better than the others
# groupbycols = ['dataset', 'ade_type', 'section', 'prompt', 'method']
# additional_querystr = " & model != 'code-llama-34b' & method == 'LLM'"

# compare ade_types
# groupbycols = ['ade_type', 'dataset', 'ade_type', 'section', 'method', 'model', 'system', 'prompt']
# additional_querystr = ""

querystr = "evaltype=='lenient' & dataset=='train' & section == 'adverse reactions'" + additional_querystr
metric = 'f1'
overall_results_df.query(querystr).groupby(groupbycols)[f"macro_{metric}"].agg(['count', 'mean', 'min', 'max'])

count  \
dataset ade_type section           method model              system          prompt           temperature          
train   all      adverse reactions LLM    code-llama-34b     fatal-prompt-v2 pharmexpert-v0   temp0            1   
                                                                             pharmexpert-v1   temp0            1   
                                          gpt-3.5-turbo-0125 fatal-prompt-v2 no-system-prompt temp0            3   
                                                                             pharmexpert-v0   temp0            3   
                                                                                              temp0.5          3   
                                                                             pharmexpert-v1   temp0            4   
                                          gpt-4-1106-preview fatal-prompt-v2 no-system-prompt temp0            3   
                                                                             pharmexpert-v0   temp0            3   
                                                                                              temp0.5          3   
                                                                                              temp1.0          3   
                                                                             pharmexpert-v1   temp0            5   
                                                             fatal-prompt-v3 pharmexpert-v1   temp0            1   
                                   exact  exact              NA              NA               NA               1   

                                                                                                               mean  \
dataset ade_type section           method model              system          prompt           temperature             
train   all      adverse reactions LLM    code-llama-34b     fatal-prompt-v2 pharmexpert-v0   temp0        0.522793   
                                                                             pharmexpert-v1   temp0        0.518157   
                                          gpt-3.5-turbo-0125 fatal-prompt-v2 no-system-prompt temp0        0.730736   
                                                                             pharmexpert-v0   temp0        0.788958   
                                                                                              temp0.5      0.789253   
                                                                             pharmexpert-v1   temp0        0.810535   
                                          gpt-4-1106-preview fatal-prompt-v2 no-system-prompt temp0        0.905270   
                                                                             pharmexpert-v0   temp0        0.903480   
                                                                                              temp0.5      0.902456   
                                                                                              temp1.0      0.902636   
                                                                             pharmexpert-v1   temp0        0.907799   
                                                             fatal-prompt-v3 pharmexpert-v1   temp0        0.893901   
                                   exact  exact              NA              NA               NA           0.802733   

                                                                                                                min  \
dataset ade_type section           method model              system          prompt           temperature             
train   all      adverse reactions LLM    code-llama-34b     fatal-prompt-v2 pharmexpert-v0   temp0        0.522793   
                                                                             pharmexpert-v1   temp0        0.518157   
                                          gpt-3.5-turbo-0125 fatal-prompt-v2 no-system-prompt temp0     

In [268]:
overall_results_df.query(querystr).groupby(groupbycols)[f"micro_{metric}"].agg(['count', 'mean', 'min', 'max'])

count  \
dataset ade_type section           method model              system          prompt           temperature          
train   all      adverse reactions LLM    code-llama-34b     fatal-prompt-v2 pharmexpert-v0   temp0            1   
                                                                             pharmexpert-v1   temp0            1   
                                          gpt-3.5-turbo-0125 fatal-prompt-v2 no-system-prompt temp0            3   
                                                                             pharmexpert-v0   temp0            3   
                                                                                              temp0.5          3   
                                                                             pharmexpert-v1   temp0            4   
                                          gpt-4-1106-preview fatal-prompt-v2 no-system-prompt temp0            3   
                                                                             pharmexpert-v0   temp0            3   
                                                                                              temp0.5          3   
                                                                                              temp1.0          3   
                                                                             pharmexpert-v1   temp0            5   
                                                             fatal-prompt-v3 pharmexpert-v1   temp0            1   
                                   exact  exact              NA              NA               NA               1   

                                                                                                               mean  \
dataset ade_type section           method model              system          prompt           temperature             
train   all      adverse reactions LLM    code-llama-34b     fatal-prompt-v2 pharmexpert-v0   temp0        0.420294   
                                                                             pharmexpert-v1   temp0        0.429824   
                                          gpt-3.5-turbo-0125 fatal-prompt-v2 no-system-prompt temp0        0.711965   
                                                                             pharmexpert-v0   temp0        0.775151   
                                                                                              temp0.5      0.778933   
                                                                             pharmexpert-v1   temp0        0.799266   
                                          gpt-4-1106-preview fatal-prompt-v2 no-system-prompt temp0        0.894240   
                                                                             pharmexpert-v0   temp0        0.893306   
                                                                                              temp0.5      0.894164   
                                                                                              temp1.0      0.892385   
                                                                             pharmexpert-v1   temp0        0.897691   
                                                             fatal-prompt-v3 pharmexpert-v1   temp0        0.884542   
                                   exact  exact              NA              NA               NA           0.835516   

                                                                                                                min  \
dataset ade_type section           method model              system          prompt           temperature             
train   all      adverse reactions LLM    code-llama-34b     fatal-prompt-v2 pharmexpert-v0   temp0        0.420294   
                                                                             pharmexpert-v1   temp0        0.429824   
                                          gpt-3.5-turbo-0125 fatal-prompt-v2 no-system-prompt temp0     